In [1]:
from nw.parser import NwParser
from sqlalchemy import create_engine
import pandas as pd
from nw.settings import NW_SQL_PATH, ELASTIC_HOSTS, ELK_HOST
# import pyelasticsearch
from nw.loggers import logger
from concurrent.futures import ProcessPoolExecutor
import joblib


def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]



# es = pyelasticsearch.ElasticSearch('http://' + ELK_HOST)
# engine = create_engine('sqlite:///{}'.format(NW_SQL_PATH))
# df = pd.read_sql('select topic_html, topic_url from nwdump limit 100', engine)
# parser = NwParser()

     

     
# failed = [] 
# p_json = []
# t_json = []
# for i, (topic_url, topic_html) in enumerate(zip(df.topic_url, df.topic_html)):
#     if i%1000 == 0:
#         print('processed {}'.format(i))
#     try:
#         posts_json, topic_json = parser.topic_html_to_json(topic_html)
#         p_json.append(posts_json)
#         t_json.append(topic_json)
        
#     except:
#         failed.append(topic_url)

In [16]:
%prun  parser.topic_html_to_json(df.topic_html.iloc[0])

In [9]:
( 170000 * 0.1 ) / 3600

4.722222222222222

In [1]:
# (item for sublist in l for item in sublist)

In [21]:
del post_df

In [22]:
posts_df = pd.DataFrame.from_records(itertools.chain(*posts))

In [26]:
posts_df.to_hdf('posts_all.h5','posts')

/home/i008/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:1101: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['cites', 'forum_id', 'post_body', 'post_id', 'unique_post_id', 'user_href', 'user_name']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [32]:
for row in posts_df.iterrows():
    a = row
    break

In [36]:
a = a[1]

In [5]:
# import pandas as pd
# df = pd.read_hdf('posts_all.h5','posts')

h = df.head()
h.to_m

In [40]:
from nw.parser import NwParser
from sqlalchemy import create_engine
import pandas as pd
from nw.settings import NW_SQL_PATH, ELASTIC_HOSTS, ELK_HOST
import pyelasticsearch
from nw.loggers import logger


def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


        

es = pyelasticsearch.ElasticSearch('http://' + ELK_HOST)
engine = create_engine('sqlite:///{}'.format(NW_SQL_PATH))
df = pd.read_sql('select  topic_html from nwdump limit 100', engine)


# docs = [es.index_op(d, id=d.get('unique_post_id')) for d in collect]
# for i, chun in enumerate(chunks(docs, int(len(docs) * .05))):
#     logger.info('processing chunk {}'.format(i))
#     res = es.bulk(chun, index='nw', doc_type='post')
#     print(res)
#     print('---' * 10)
#     print(failed)

In [13]:
df['post_length'] = df.post_body.apply(lambda x: len(x))

In [2]:
# df[df.user_name.isin(df.user_name.value_counts()[:2000].index)].groupby(df.user_name)['post_length'].mean().sort_values(ascending=False)

In [30]:
len(df.post_body.loc[10])

119

In [8]:
df.shape

(5496160, 9)

In [9]:
import joblib

In [10]:
topics_=joblib.load('t_json.pckl')

In [12]:
df_topics = pd.DataFrame.from_records(topics_)

In [13]:
df_topics.to_hdf('topics_all.h5','topics')

In [3]:
from nw.jobs import index_topics

In [4]:
index_topics([173498])

2016-12-01 19:43:07,192 - nw-logger - INFO - RUNNING RE-INDEX-SCRAPE JOB for topics: [173498]
2016-12-01 19:43:07,195 - nw-logger - INFO - Indexing topic 173498
2016-12-01 19:43:07,198 - nw-logger - DEBUG - parsing topic number 173498
2016-12-01 19:43:08,048 - nw-logger - INFO - {'forum_id': '/forum/4', 'topic_name': '#PizzaGate', 'topic_id': 173498, 'post_date': datetime.datetime(2016, 11, 20, 22, 39, 19)}
2016-12-01 19:43:08,049 - nw-logger - INFO - ['{"index": {"_id": 173498}}\n{"forum_id": "/forum/4", "topic_name": "#PizzaGate", "topic_id": 173498, "post_date": "2016-11-20T22:39:19"}']


1

In [6]:
from pyelasticsearch import ElasticSearch

In [7]:
es = ElasticSearch()

In [ ]:
es.search(
{
    "sort" : 
        { "topic_id" : {"order" : "asc"}},

}

)

In [6]:
import pandas as pd

In [7]:
posts = pd.read_hdf('/media/i008/duzy1/posts_all.h5')
topic = pd.read_hdf('/media/i008/duzy1/topics_all.h5')

In [12]:
topic.shape

(144639, 4)

In [8]:
M = posts.merge(topic[['topic_id','topic_name']], on='topic_id')

In [21]:
from concurrent.futures import ProcessPoolExecutor
import sqlite3
from nw.loggers import logger
from nw.parser import NwParser
import datetime
import joblib
import pandas as pd
from pandas.io import sql
import json


parser = NwParser()
conn =sqlite3.connect('/media/i008/duzy1/nwdb.sqlite')

def t_to_json(html):
    try:
        return parser.topic_html_to_json(html)
    except:
#         logger.info('--failed--')
        pass

select = conn.execute('select topic_html from nwdump')
collect = []
ii = 0
for row in select:
    if not ii%1000: 
        logger.info('{} -- {}'.format(datetime.datetime.utcnow(), ii))       
    collect.append(t_to_json(row[0]))
    ii+=1

    
joblib.dump(collect, 'all_posts_list.pickl')


df = pd.DataFrame.from_records(flattened)
df.to_hdf('allposts_new','posts')


dx = df[:]
dx.cites = dx.cites.apply(lambda x: json.dumps(x))
dx.post_date = dx.post_date.astype(str)
cnx = sqlite3.connect('nw_posts.sqlite')
sql.to_sql(dx,'nw_posts', cnx, if_exists='replace', chunksize=50000)

In [25]:
# flattened = [item for sublist in filter(None,collect) for item in sublist]
joblib.dump(flattened, 'flattened_posts.pickl')


['flattened_posts.pickl']

In [39]:
d = pd.read_sql('select * from nw_posts', cnx)

In [3]:
# %matplotlib inline
# d.user_name.value_counts()[:30].plot(kind='bar')

In [1]:
import pandas as pd

In [2]:
df = pd.read_hdf('/media/i008/duzy1/nw_data/nw_posts.hdf5', 'posts')

In [4]:
# df.head().to_html()